Importamos as bibliteca **pandas** e **matplotlib.pyplot** para auxiliar nos processos de manipulação e análise de dados e criação de gráficos e visualizações de dados, respectivamente.

In [79]:
import pandas
import numpy
import matplotlib.pyplot

Para realizar a leitura da base de dados, atribuimos a variável `data`:

In [80]:
# leitura do arquivo .csv
data = pandas.read_csv('PS_20174392719_1491204439457_log.csv')

Para listar as transações em grupos distintos, com a existência ou não de fraude, utilizamos a seguinte atribuição:

In [81]:
fraud_transactions     = data.loc[data['isFraud'] == 1]
non_fraud_transactions = data.loc[data['isFraud'] == 0]

Dado que o campo de nome descreve o tipo de conta envolvida na transação, precisamos saber o quão relevantes eles são para o problema. Para isso, faremos os seguintes testes:

In [85]:
amount_merchant_orig = len(data.loc[data['nameOrig'].str.match('M')])
amount_merchant_dest = len(data.loc[data['nameDest'].str.match('M')])
amount_fraud_orig_merchant = len(fraud_transactions.loc[fraud_transactions['nameOrig'].str.match('M')])
amount_fraud_dest_merchant = len(fraud_transactions.loc[fraud_transactions['nameDest'].str.match('M')])
amount_non_fraud_orig_merchant = len(non_fraud_transactions.loc[non_fraud_transactions['nameOrig'].str.match('M')])
amount_non_fraud_dest_merchant = len(non_fraud_transactions.loc[non_fraud_transactions['nameDest'].str.match('M')])

print('Quantidade de transações originadas de comerciantes:', amount_merchant_orig)
print('Quantidade de transações destinadas a comerciantes:', amount_merchant_dest)
print('Quantidade de transações fraudulentas originadas de comerciantes:', amount_fraud_orig_merchant)
print('Quantidade de transações fraudulentas destinadas a comerciantes:', amount_fraud_dest_merchant)
print('Quantidade de transações não fraudulentas originadas de comerciantes:', amount_non_fraud_orig_merchant)
print('Quantidade de transações não fraudulentas destinadas a comerciantes:', amount_non_fraud_dest_merchant)

Quantidade de transações originadas de comerciantes: 0
Quantidade de transações destinadas a comerciantes: 2151495
Quantidade de transações fraudulentas originadas de comerciantes: 0
Quantidade de transações fraudulentas destinadas a comerciantes: 0
Quantidade de transações não fraudulentas originadas de comerciantes: 0
Quantidade de transações não fraudulentas destinadas a comerciantes: 2151495


Podemos verificar que em todas as transações envolvendo comerciantes,s, eles aparecem como destinatári e em nenhuma ocorre fraude. 
Essa informação é interessante para a árvore de decisão, então vamos simplificar os dados removendo as colunas de nome e adicionando uma nova coluna indicando quando uma transação envolve um comerciante.

In [84]:
data['hasMerchantAccount'] = numpy.where(data['nameDest'].str.match('M'), 1, 0)
reduced_data = data.drop(columns=['nameOrig', 'nameDest'])
reduced_data.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,hasMerchantAccount
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0,1
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0,1
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0,0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0,0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0,1
